# Milestone 3: Finetune the model with sidewalks dataset

First, please manually copy the extracted sidewalks data to the current folder.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
import random
from scipy import ndimage
from pathlib import Path

Plot a sample test file and the label